## Tools

In [ ]:
!pip install datasets GPUtil bitsandbytes

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 MB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 108.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 86.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 39.8 MB/s eta 0:00

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt

from datasets import load_dataset
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    DistilBertForSequenceClassification,
    Trainer,
    TrainingArguments,
    TrainerCallback
)
import numpy as np
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    confusion_matrix,
    roc_curve,
    auc,
    f1_score
)
import psutil
import GPUtil
import pandas as pd

In [ ]:
from huggingface_hub import login

login(token="HUGGING_FACE_KEY")

## Network Intrusion Dataset

In [ ]:
from datasets import load_dataset

dataset = load_dataset("gates04/network-intrusion-dataset")

train_dataset = dataset["train"]
validation_dataset = dataset["validation"]
test_dataset = dataset["test"]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/586 [00:00<?, ?B/s]

train-00000-of-00005.parquet:   0%|          | 0.00/135M [00:00<?, ?B/s]

train-00001-of-00005.parquet:   0%|          | 0.00/135M [00:00<?, ?B/s]

train-00002-of-00005.parquet:   0%|          | 0.00/135M [00:00<?, ?B/s]

...

validation-00000-of-00001.parquet:   0%|          | 0.00/145M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/145M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1187781 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/254525 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/254525 [00:00<?, ? examples/s]

## Teacher Model (DistilBERT) && Student Model (TinyBERT) for Knowledge Distillation

In [ ]:
from transformers import BitsAndBytesConfig

teacher_model_name = "rdpahalavan/bert-network-packet-flow-header-payload"
teacher_tokenizer = AutoTokenizer.from_pretrained(teacher_model_name)
teacher_model = AutoModelForSequenceClassification.from_pretrained(teacher_model_name)

teacher_model.eval()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
teacher_model.to(device)

student_model_name = "huawei-noah/TinyBERT_General_4L_312D"
student_tokenizer = AutoTokenizer.from_pretrained(student_model_name)
student_model = DistilBertForSequenceClassification.from_pretrained(student_model_name, num_labels=24)
student_model.to(device)

tokenizer_config.json:   0%|          | 0.00/321 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/669k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.78k [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

You are using a model of type bert to instantiate a model of type distilbert. This is not supported for all configurations of models and can yield errors.


pytorch_model.bin:   0%|          | 0.00/62.7M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at huawei-noah/TinyBERT_General_4L_312D and are newly initialized: ['classifier.bias', 'classifier.weight', 'distilbert.embeddings.LayerNorm.bias', 'distilbert.embeddings.LayerNorm.weight', 'distilbert.embeddings.position_embeddings.weight', 'distilbert.embeddings.word_embeddings.weight', 'distilbert.transformer.layer.0.attention.k_lin.bias', 'distilbert.transformer.layer.0.attention.k_lin.weight', 'distilbert.transformer.layer.0.attention.out_lin.bias', 'distilbert.transformer.layer.0.attention.out_lin.weight', 'distilbert.transformer.layer.0.attention.q_lin.bias', 'distilbert.transformer.layer.0.attention.q_lin.weight', 'distilbert.transformer.layer.0.attention.v_lin.bias', 'distilbert.transformer.layer.0.attention.v_lin.weight', 'distilbert.transformer.layer.0.ffn.lin1.bias', 'distilbert.transformer.layer.0.ffn.lin1.weight', 'distilbert.transformer.layer.0.ffn.lin2.bias', 'distilbert.t

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 312, padding_idx=0)
      (position_embeddings): Embedding(512, 312)
      (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-3): 4 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=312, out_features=312, bias=True)
            (k_lin): Linear(in_features=312, out_features=312, bias=True)
            (v_lin): Linear(in_features=312, out_features=312, bias=True)
            (out_lin): Linear(in_features=312, out_features=312, bias=True)
          )
          (sa_layer_norm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


## Model for Self Distillation

In [ ]:
from transformers import AutoConfig

model_name = "gates04/FP16-KD-NID"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

labels = [
    'Analysis', 'Backdoor', 'Bot', 'DDoS', 'DoS', 'DoS GoldenEye',
    'DoS Hulk', 'DoS SlowHTTPTest', 'DoS Slowloris', 'Exploits',
    'FTP Patator', 'Fuzzers', 'Generic', 'Heartbleed', 'Infiltration',
    'Normal', 'Port Scan', 'Reconnaissance', 'SSH Patator',
    'Shellcode', 'Web Attack - Brute Force', 'Web Attack - SQL Injection',
    'Web Attack - XSS', 'Worms'
]

config = AutoConfig.from_pretrained(model_name)

id2label = {str(i): label for i, label in enumerate(labels)}
label2id = {label: i for i, label in enumerate(labels)}

config.id2label = id2label
config.label2id = label2id

model.eval()

tokenizer_config.json:   0%|          | 0.00/1.33k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/669k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.78k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/76.2M [00:00<?, ?B/s]

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 312, padding_idx=0)
      (position_embeddings): Embedding(512, 312)
      (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-3): 4 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=312, out_features=312, bias=True)
            (k_lin): Linear(in_features=312, out_features=312, bias=True)
            (v_lin): Linear(in_features=312, out_features=312, bias=True)
            (out_lin): Linear(in_features=312, out_features=312, bias=True)
          )
          (sa_layer_norm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


In [ ]:
from transformers import AutoConfig

model_name = "gates04/Structured-FP16-KD-NID"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

labels = [
    'Analysis', 'Backdoor', 'Bot', 'DDoS', 'DoS', 'DoS GoldenEye',
    'DoS Hulk', 'DoS SlowHTTPTest', 'DoS Slowloris', 'Exploits',
    'FTP Patator', 'Fuzzers', 'Generic', 'Heartbleed', 'Infiltration',
    'Normal', 'Port Scan', 'Reconnaissance', 'SSH Patator',
    'Shellcode', 'Web Attack - Brute Force', 'Web Attack - SQL Injection',
    'Web Attack - XSS', 'Worms'
]


config = AutoConfig.from_pretrained(model_name)

id2label = {str(i): label for i, label in enumerate(labels)}
label2id = {label: i for i, label in enumerate(labels)}

config.id2label = id2label
config.label2id = label2id

model.eval()

tokenizer_config.json:   0%|          | 0.00/1.33k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/669k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.78k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/76.2M [00:00<?, ?B/s]

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 312, padding_idx=0)
      (position_embeddings): Embedding(512, 312)
      (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-3): 4 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=312, out_features=312, bias=True)
            (k_lin): Linear(in_features=312, out_features=312, bias=True)
            (v_lin): Linear(in_features=312, out_features=312, bias=True)
            (out_lin): Linear(in_features=312, out_features=312, bias=True)
          )
          (sa_layer_norm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


In [ ]:
from transformers import AutoConfig

model_name = "gates04/Structured-NF4-KD-NID"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

labels = [
    'Analysis', 'Backdoor', 'Bot', 'DDoS', 'DoS', 'DoS GoldenEye',
    'DoS Hulk', 'DoS SlowHTTPTest', 'DoS Slowloris', 'Exploits',
    'FTP Patator', 'Fuzzers', 'Generic', 'Heartbleed', 'Infiltration',
    'Normal', 'Port Scan', 'Reconnaissance', 'SSH Patator',
    'Shellcode', 'Web Attack - Brute Force', 'Web Attack - SQL Injection',
    'Web Attack - XSS', 'Worms'
]

config = AutoConfig.from_pretrained(model_name)

id2label = {str(i): label for i, label in enumerate(labels)}
label2id = {label: i for i, label in enumerate(labels)}

config.id2label = id2label
config.label2id = label2id

model.eval()

tokenizer_config.json:   0%|          | 0.00/1.33k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/669k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/76.2M [00:00<?, ?B/s]

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 312, padding_idx=0)
      (position_embeddings): Embedding(512, 312)
      (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-3): 4 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=312, out_features=312, bias=True)
            (k_lin): Linear(in_features=312, out_features=312, bias=True)
            (v_lin): Linear(in_features=312, out_features=312, bias=True)
            (out_lin): Linear(in_features=312, out_features=312, bias=True)
          )
          (sa_layer_norm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


In [ ]:
from transformers import AutoConfig

model_name = "gates04/Structured-FP16-KD-SD-NID"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

labels = [
    'Analysis', 'Backdoor', 'Bot', 'DDoS', 'DoS', 'DoS GoldenEye',
    'DoS Hulk', 'DoS SlowHTTPTest', 'DoS Slowloris', 'Exploits',
    'FTP Patator', 'Fuzzers', 'Generic', 'Heartbleed', 'Infiltration',
    'Normal', 'Port Scan', 'Reconnaissance', 'SSH Patator',
    'Shellcode', 'Web Attack - Brute Force', 'Web Attack - SQL Injection',
    'Web Attack - XSS', 'Worms'
]

config = AutoConfig.from_pretrained(model_name)

id2label = {str(i): label for i, label in enumerate(labels)}
label2id = {label: i for i, label in enumerate(labels)}

config.id2label = id2label
config.label2id = label2id

model.eval()

tokenizer_config.json:   0%|          | 0.00/1.33k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/669k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/76.2M [00:00<?, ?B/s]

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 312, padding_idx=0)
      (position_embeddings): Embedding(512, 312)
      (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-3): 4 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=312, out_features=312, bias=True)
            (k_lin): Linear(in_features=312, out_features=312, bias=True)
            (v_lin): Linear(in_features=312, out_features=312, bias=True)
            (out_lin): Linear(in_features=312, out_features=312, bias=True)
          )
          (sa_layer_norm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


## Tokenization

In [ ]:
from transformers import AutoConfig

if hasattr(config, "label2id"):
    label_mapping = config.label2id
    reverse_label_mapping = {v: k for k, v in label_mapping.items()}
else:
    label_mapping = {"Normal": 0, "DoS": 1, "Probe": 2, "U2R": 3, "R2L": 4}  # Örnek!
    reverse_label_mapping = {v: k for k, v in label_mapping.items()}

print("Label Mapping:", label_mapping)

Label Mapping: {'Analysis': 0, 'Backdoor': 1, 'Bot': 2, 'DDoS': 3, 'DoS': 4, 'DoS GoldenEye': 5, 'DoS Hulk': 6, 'DoS SlowHTTPTest': 7, 'DoS Slowloris': 8, 'Exploits': 9, 'FTP Patator': 10, 'Fuzzers': 11, 'Generic': 12, 'Heartbleed': 13, 'Infiltration': 14, 'Normal': 15, 'Port Scan': 16, 'Reconnaissance': 17, 'SSH Patator': 18, 'Shellcode': 19, 'Web Attack - Brute Force': 20, 'Web Attack - SQL Injection': 21, 'Web Attack - XSS': 22, 'Worms': 23}


In [ ]:
max_length = 512

def tokenize_dataset(examples):
    texts = examples["packet_dat"]
    tokenized_inputs = tokenizer(texts, padding="max_length", truncation=True, max_length=max_length)
    tokenized_inputs["labels"] = [label_mapping[label] for label in examples["attack_cat"]]
    return tokenized_inputs


train_encoded = train_dataset.map(tokenize_dataset, batched=True, num_proc=16)
val_encoded   = validation_dataset.map(tokenize_dataset, batched=True, num_proc=16)
test_encoded  = test_dataset.map(tokenize_dataset, batched=True, num_proc=16)

train_encoded.set_format("torch", columns=["input_ids", "attention_mask", "labels"])
val_encoded.set_format("torch", columns=["input_ids", "attention_mask", "labels"])
test_encoded.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

train_encoded = train_encoded.remove_columns(["packet_dat", "attack_cat"])
val_encoded = val_encoded.remove_columns(["packet_dat", "attack_cat"])
test_encoded = test_encoded.remove_columns(["packet_dat", "attack_cat"])

Map (num_proc=16):   0%|          | 0/1187781 [00:00<?, ? examples/s]

Map (num_proc=16):   0%|          | 0/254525 [00:00<?, ? examples/s]

Map (num_proc=16):   0%|          | 0/254525 [00:00<?, ? examples/s]

## Knowledge Distillation Loss Function

In [ ]:
def distillation_loss(student_logits, teacher_logits, labels, alpha=0.5, temperature=2.0):

    ce_loss = nn.CrossEntropyLoss()(student_logits, labels)

    t_logits = teacher_logits / temperature
    s_logits = student_logits / temperature

    teacher_probs = F.softmax(t_logits, dim=-1)
    student_log_probs = F.log_softmax(s_logits, dim=-1)

    kld_loss = F.kl_div(student_log_probs, teacher_probs, reduction="batchmean") * (temperature**2)
    return alpha * kld_loss + (1.0 - alpha) * ce_loss

## Knowledge Distillation Trainer

In [ ]:
class DistillationTrainer(Trainer):
    def __init__(self, teacher_model, alpha=0.5, temperature=2.0, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.teacher_model = teacher_model
        self.alpha = alpha
        self.temperature = temperature
        self.teacher_model.eval()

    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        labels = inputs.get("labels")

        # Student forward pass
        outputs_student = model(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"]
        )
        student_logits = outputs_student.logits

        # Teacher forward pass (no grad)
        with torch.no_grad():
            outputs_teacher = self.teacher_model(
                input_ids=inputs["input_ids"],
                attention_mask=inputs["attention_mask"]
            )
        teacher_logits = outputs_teacher.logits

        # Distillation
        loss = distillation_loss(
            student_logits=student_logits,
            teacher_logits=teacher_logits,
            labels=labels,
            alpha=self.alpha,
            temperature=self.temperature
        )
        return (loss, outputs_student) if return_outputs else loss

## Self Distillation Trainer

In [ ]:
import copy

class SelfDistillationTrainer(Trainer):

    def __init__(
        self,
        model,
        alpha=0.5,
        temperature=2.0,
        use_ema_teacher=True,
        ema_decay=0.995,
        *args,
        **kwargs
    ):
        super().__init__(model, *args, **kwargs)
        self.alpha = alpha
        self.temperature = temperature
        self.use_ema_teacher = use_ema_teacher
        self.ema_decay = ema_decay

        self.teacher_model = copy.deepcopy(model)
        self.teacher_model.eval()

    def ema_update(self):

        for teacher_param, student_param in zip(self.teacher_model.parameters(), self.model.parameters()):
            teacher_param.data = self.ema_decay * teacher_param.data + (1.0 - self.ema_decay) * student_param.data

    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        labels = inputs.get("labels")

        # Student forward
        outputs_student = model(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"]
        )
        student_logits = outputs_student.logits

        # Teacher forward (no grad)
        with torch.no_grad():
            outputs_teacher = self.teacher_model(
                input_ids=inputs["input_ids"],
                attention_mask=inputs["attention_mask"]
            )
        teacher_logits = outputs_teacher.logits

        # Distillation loss
        loss = distillation_loss(
            student_logits=student_logits,
            teacher_logits=teacher_logits,
            labels=labels,
            alpha=self.alpha,
            temperature=self.temperature
        )

        return (loss, outputs_student) if return_outputs else loss

    def training_step(self, *args, **kwargs):

        loss = super().training_step(*args, **kwargs)
        if self.use_ema_teacher:
            self.ema_update()
        return loss


## System Metrics Callback

In [ ]:
class SystemMetricsCallback(TrainerCallback):
    def __init__(self):
        super().__init__()
        self.steps = []
        self.cpu_usage = []
        self.gpu_usage = []
        self.gpu_mem = []

    def on_step_end(self, args, state, control, **kwargs):
        self.steps.append(state.global_step)
        cpu_percent = psutil.cpu_percent()
        self.cpu_usage.append(cpu_percent)

        gpus = GPUtil.getGPUs()
        if gpus:
            gpu = gpus[0]
            self.gpu_usage.append(gpu.load * 100.0)
            self.gpu_mem.append(gpu.memoryUsed)
        else:
            self.gpu_usage.append(0.0)
            self.gpu_mem.append(0.0)

    def plot_system_metrics(self):

        plt.figure(figsize=(8,6))
        plt.plot(self.steps, self.cpu_usage, label="CPU Usage (%)", color="blue")
        plt.plot(self.steps, self.gpu_usage, label="GPU Usage (%)", color="red")
        plt.xlabel("Global Step")
        plt.ylabel("Usage (%)")
        plt.title("CPU and GPU Usage During Training")
        plt.legend()
        plt.show()

        plt.figure(figsize=(8,6))
        plt.plot(self.steps, self.gpu_mem, label="GPU Memory (MB)", color="green")
        plt.xlabel("Global Step")
        plt.ylabel("Memory (MB)")
        plt.title("GPU Memory Usage During Training")
        plt.legend()
        plt.show()

## Loss Metrics Callback

In [ ]:
class LossMetricsCallback(TrainerCallback):
    def __init__(self):
        super().__init__()
        self.train_loss = []
        self.eval_loss = []
        self.steps = []

    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs is not None:
            if "loss" in logs:
                self.train_loss.append(logs["loss"])
                self.steps.append(state.global_step)
            if "eval_loss" in logs:
                self.eval_loss.append(logs["eval_loss"])

    def plot_loss_curves(self):
        plt.figure(figsize=(8,6))
        plt.plot(self.steps, self.train_loss, label="Train Loss", marker="o")

        eval_x = self.steps[:len(self.eval_loss)]
        plt.plot(eval_x, self.eval_loss, label="Eval Loss", marker="x")

        plt.xlabel("Global Steps")
        plt.ylabel("Loss")
        plt.title("Training & Validation Loss")
        plt.legend()
        plt.show()

## Callbacks Initialization

In [ ]:
system_metrics_callback = SystemMetricsCallback()
loss_metrics_callback   = LossMetricsCallback()

## Training Arguments for Knowledge Distillation

In [ ]:
training_args = TrainingArguments(
    output_dir="./Structured-NF4-KD-NID",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=650,
    per_device_eval_batch_size=650,
    num_train_epochs=20,
    weight_decay=0.01,
    logging_steps=50,
    fp16=True,
    dataloader_num_workers=64,
    load_best_model_at_end=True,
    metric_for_best_model="F1 score",
    greater_is_better=True,
    report_to="none",  # no wandb
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


## Training Arguments for Self Distillation

In [ ]:
    training_args = TrainingArguments(
        output_dir="./Structured-FP16-KD-SD-new-NID",
        evaluation_strategy="epoch",
        save_strategy="epoch",
        learning_rate=5e-5,
        per_device_train_batch_size=300,
        per_device_eval_batch_size=300,
        num_train_epochs=5,
        weight_decay=0.01,
        logging_steps=50,
        fp16=True,  # Mixed precision
        dataloader_num_workers=64,
        load_best_model_at_end=True,
        metric_for_best_model="F1 score",
        greater_is_better=True,
        report_to="none",
    )

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


## Self Distillation Trainer Initialization

In [ ]:
     trainer = SelfDistillationTrainer(
        model=model,
        args=training_args,
        train_dataset=train_encoded,
        eval_dataset=val_encoded,
        alpha=0.5,
        temperature=2.0,
        use_ema_teacher=True,
        ema_decay=0.995,
        callbacks=[system_metrics_callback, loss_metrics_callback]
    )

## Knowledge Distillation Trainer Initialization

In [ ]:
!pip install torch-pruning

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.1/64.1 kB 5.5 MB/s eta 0:00:00


In [ ]:
import torch.nn.utils.prune as prune

prune.ln_structured(
    module=teacher_model.classifier,
    name="weight",
    amount=0.2,
    n=1,
    dim=1
)
prune.remove(teacher_model.classifier, "weight")

Linear(in_features=768, out_features=24, bias=True)

In [ ]:
teacher_model_fp16 = teacher_model.half()
teacher_model_fp16.eval()
teacher_model_fp16.to(device)

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


In [ ]:
from transformers import BitsAndBytesConfig

bnb_config_int4 = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)


teacher_model_int4 = AutoModelForSequenceClassification.from_pretrained(
    teacher_model_name, quantization_config=bnb_config_int4, device_map="cuda"
)

In [ ]:
teacher_model_int4.eval()
teacher_model_int4.to(device)

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear4bit(in_features=768, out_features=768, bias=True)
            (k_lin): Linear4bit(in_features=768, out_features=768, bias=True)
            (v_lin): Linear4bit(in_features=768, out_features=768, bias=True)
            (out_lin): Linear4bit(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1,

In [ ]:
trainer = DistillationTrainer(
    teacher_model=teacher_model_int4,
    alpha=0.5,
    temperature=2.0,
    model=student_model,
    args=training_args,
    train_dataset=train_encoded,
    eval_dataset=val_encoded,
    callbacks=[system_metrics_callback, loss_metrics_callback]
)

## Metrics Computation (Accuracy, Precision, Recall, FPR)

In [ ]:
def compute_metrics(eval_pred):

    logits, labels = eval_pred
    preds = logits.argmax(axis=-1)

    accuracy = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average='macro')
    precision = precision_score(labels, preds, average='macro', zero_division=0)
    recall = recall_score(labels, preds, average='macro', zero_division=0)

    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "F1 score": f1
    }

trainer.compute_metrics = compute_metrics

## Student Model Self Distillation

In [ ]:
trainer.train()

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 64 worker processes in total. Our suggested max number of worker in current system is 12, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1 score
1,0.020000,0.020042,0.991887,0.975681,0.943396,0.949857
2,0.016700,0.018013,0.992244,0.976296,0.954959,0.960872


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 64 worker processes in total. Our suggested max number of worker in current system is 12, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 64 worker processes in total. Our suggested max number of worker in current system is 12, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarnin

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1 score
1,0.020000,0.020042,0.991887,0.975681,0.943396,0.949857
2,0.016700,0.018013,0.992244,0.976296,0.954959,0.960872
3,0.013800,0.016716,0.992559,0.971286,0.963919,0.966288
4,0.012900,0.015855,0.992751,0.965390,0.959998,0.961631
5,0.010500,0.015215,0.992806,0.961226,0.966866,0.962923


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 64 worker processes in total. Our suggested max number of worker in current system is 12, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 64 worker processes in total. Our suggested max number of worker in current system is 12, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarnin

TrainOutput(global_step=19800, training_loss=0.014812141998548701, metrics={'train_runtime': 11268.7528, 'train_samples_per_second': 527.024, 'train_steps_per_second': 1.757, 'total_flos': 1.7066696635957248e+17, 'train_loss': 0.014812141998548701, 'epoch': 5.0})

In [ ]:
trainer.train()

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 64 worker processes in total. Our suggested max number of worker in current system is 12, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1 score
1,0.029700,0.022320,0.991235,0.932478,0.915458,0.918511


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 64 worker processes in total. Our suggested max number of worker in current system is 12, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 64 worker processes in total. Our suggested max number of worker in current system is 12, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [ ]:
trainer.train()

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 64 worker processes in total. Our suggested max number of worker in current system is 12, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1 score
1,0.022400,0.020882,0.991466,0.931000,0.930756,0.929811
2,0.020300,0.019842,0.991781,0.930840,0.930627,0.928890
3,0.016300,0.018559,0.991887,0.961237,0.942297,0.946796
4,0.016800,0.019461,0.991950,0.935082,0.920001,0.924537
5,0.013200,0.017021,0.992292,0.962527,0.959508,0.960313


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 64 worker processes in total. Our suggested max number of worker in current system is 12, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 64 worker processes in total. Our suggested max number of worker in current system is 12, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarnin

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1 score
1,0.022400,0.020882,0.991466,0.931000,0.930756,0.929811
2,0.020300,0.019842,0.991781,0.930840,0.930627,0.928890
3,0.016300,0.018559,0.991887,0.961237,0.942297,0.946796
4,0.016800,0.019461,0.991950,0.935082,0.920001,0.924537
5,0.013200,0.017021,0.992292,0.962527,0.959508,0.960313
6,0.015400,0.016867,0.992433,0.969409,0.960308,0.962992
7,0.010500,0.016533,0.992472,0.970082,0.959027,0.962739
8,0.013300,0.016129,0.992661,0.966070,0.966253,0.964981
9,0.013000,0.015850,0.992692,0.978233,0.967659,0.971143
10,0.011200,0.015609,0.992732,0.963149,0.967768,0.964570


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 64 worker processes in total. Our suggested max number of worker in current system is 12, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 64 worker processes in total. Our suggested max number of worker in current system is 12, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarnin

TrainOutput(global_step=23760, training_loss=0.015710366655592667, metrics={'train_runtime': 9134.0624, 'train_samples_per_second': 1300.386, 'train_steps_per_second': 2.601, 'total_flos': 3.4133393271914496e+17, 'train_loss': 0.015710366655592667, 'epoch': 10.0})

## Student Model Training && Validation

In [ ]:
trainer.train()

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 64 worker processes in total. Our suggested max number of worker in current system is 12, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1 score
1,0.217000,0.185933,0.977460,0.780807,0.781113,0.779659
2,0.129500,0.114208,0.984355,0.914213,0.842124,0.840406
3,0.095800,0.091597,0.986441,0.913595,0.866189,0.869935
4,0.084600,0.077892,0.988213,0.920653,0.890219,0.895336
5,0.068300,0.067743,0.989439,0.924572,0.903379,0.907319
6,0.070000,0.059853,0.989915,0.925853,0.910291,0.911786
7,0.057400,0.055472,0.990425,0.928434,0.909718,0.915615
8,0.050300,0.054141,0.990559,0.921071,0.918535,0.916199
9,0.048000,0.049759,0.991121,0.931790,0.918502,0.921662
10,0.047900,0.048276,0.991239,0.935317,0.917813,0.923191


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 64 worker processes in total. Our suggested max number of worker in current system is 12, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 64 worker processes in total. Our suggested max number of worker in current system is 12, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarnin

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1 score
1,0.217000,0.185933,0.977460,0.780807,0.781113,0.779659
2,0.129500,0.114208,0.984355,0.914213,0.842124,0.840406
3,0.095800,0.091597,0.986441,0.913595,0.866189,0.869935
4,0.084600,0.077892,0.988213,0.920653,0.890219,0.895336
5,0.068300,0.067743,0.989439,0.924572,0.903379,0.907319
6,0.070000,0.059853,0.989915,0.925853,0.910291,0.911786
7,0.057400,0.055472,0.990425,0.928434,0.909718,0.915615
8,0.050300,0.054141,0.990559,0.921071,0.918535,0.916199
9,0.048000,0.049759,0.991121,0.931790,0.918502,0.921662
10,0.047900,0.048276,0.991239,0.935317,0.917813,0.923191


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 64 worker processes in total. Our suggested max number of worker in current system is 12, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 64 worker processes in total. Our suggested max number of worker in current system is 12, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarnin

TrainOutput(global_step=36560, training_loss=0.10100663633654362, metrics={'train_runtime': 22402.5685, 'train_samples_per_second': 1060.397, 'train_steps_per_second': 1.632, 'total_flos': 6.826678654382899e+17, 'train_loss': 0.10100663633654362, 'epoch': 20.0})

In [ ]:
trainer.train()

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 64 worker processes in total. Our suggested max number of worker in current system is 12, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1 score
1,0.150900,0.122835,0.982992,0.865848,0.839166,0.834319
2,0.096100,0.088350,0.986461,0.910994,0.867245,0.865174
3,0.081200,0.076322,0.987875,0.915228,0.886933,0.888187
4,0.073000,0.065002,0.988885,0.925249,0.896571,0.900693
5,0.060500,0.056788,0.989671,0.927220,0.902631,0.907466
6,0.057100,0.051580,0.990186,0.913062,0.920216,0.915570
7,0.048700,0.046033,0.990877,0.928216,0.922824,0.924748
8,0.045000,0.047055,0.990677,0.921899,0.920804,0.919607
9,0.039600,0.044301,0.991015,0.927871,0.925272,0.925763
10,0.040900,0.042173,0.991294,0.926935,0.931520,0.928829


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 64 worker processes in total. Our suggested max number of worker in current system is 12, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 64 worker processes in total. Our suggested max number of worker in current system is 12, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarnin

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1 score
1,0.150900,0.122835,0.982992,0.865848,0.839166,0.834319
2,0.096100,0.088350,0.986461,0.910994,0.867245,0.865174
3,0.081200,0.076322,0.987875,0.915228,0.886933,0.888187
4,0.073000,0.065002,0.988885,0.925249,0.896571,0.900693
5,0.060500,0.056788,0.989671,0.927220,0.902631,0.907466
6,0.057100,0.051580,0.990186,0.913062,0.920216,0.915570
7,0.048700,0.046033,0.990877,0.928216,0.922824,0.924748
8,0.045000,0.047055,0.990677,0.921899,0.920804,0.919607
9,0.039600,0.044301,0.991015,0.927871,0.925272,0.925763
10,0.040900,0.042173,0.991294,0.926935,0.931520,0.928829


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 64 worker processes in total. Our suggested max number of worker in current system is 12, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 64 worker processes in total. Our suggested max number of worker in current system is 12, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarnin

TrainOutput(global_step=36560, training_loss=0.05782456563814203, metrics={'train_runtime': 22701.9125, 'train_samples_per_second': 1046.415, 'train_steps_per_second': 1.61, 'total_flos': 6.826678654382899e+17, 'train_loss': 0.05782456563814203, 'epoch': 20.0})

In [ ]:
trainer.train()

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 64 worker processes in total. Our suggested max number of worker in current system is 12, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1 score
1,0.211200,0.182414,0.977487,0.775922,0.778406,0.774197
2,0.119600,0.106112,0.984960,0.906047,0.852169,0.850208
3,0.095600,0.085464,0.987400,0.913982,0.883320,0.885813
4,0.082300,0.070765,0.989125,0.920395,0.898158,0.902888
5,0.065300,0.062829,0.989734,0.931262,0.898463,0.907024
6,0.066000,0.057230,0.990135,0.920596,0.913245,0.910573
7,0.054600,0.053413,0.990669,0.928429,0.915608,0.918715
8,0.048600,0.051073,0.990932,0.928669,0.918390,0.919907
9,0.047300,0.049017,0.991125,0.973227,0.928382,0.934650
10,0.048300,0.046925,0.991384,0.978810,0.923748,0.934354


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 64 worker processes in total. Our suggested max number of worker in current system is 12, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 64 worker processes in total. Our suggested max number of worker in current system is 12, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarnin

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1 score
1,0.211200,0.182414,0.977487,0.775922,0.778406,0.774197
2,0.119600,0.106112,0.984960,0.906047,0.852169,0.850208
3,0.095600,0.085464,0.987400,0.913982,0.883320,0.885813
4,0.082300,0.070765,0.989125,0.920395,0.898158,0.902888
5,0.065300,0.062829,0.989734,0.931262,0.898463,0.907024
6,0.066000,0.057230,0.990135,0.920596,0.913245,0.910573
7,0.054600,0.053413,0.990669,0.928429,0.915608,0.918715
8,0.048600,0.051073,0.990932,0.928669,0.918390,0.919907
9,0.047300,0.049017,0.991125,0.973227,0.928382,0.934650
10,0.048300,0.046925,0.991384,0.978810,0.923748,0.934354


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 64 worker processes in total. Our suggested max number of worker in current system is 12, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 64 worker processes in total. Our suggested max number of worker in current system is 12, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarnin

TrainOutput(global_step=27420, training_loss=0.10546667625615407, metrics={'train_runtime': 17029.668, 'train_samples_per_second': 1046.216, 'train_steps_per_second': 1.61, 'total_flos': 5.1200089907871744e+17, 'train_loss': 0.10546667625615407, 'epoch': 15.0})

In [ ]:
trainer.train()

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 64 worker processes in total. Our suggested max number of worker in current system is 12, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1 score
1,0.158200,0.132869,0.982670,0.872999,0.832117,0.830746
2,0.111200,0.103264,0.985345,0.869651,0.858425,0.852904
3,0.092700,0.082014,0.987872,0.920866,0.879031,0.878888
4,0.082800,0.068862,0.989266,0.923323,0.895283,0.898007
5,0.071100,0.063718,0.989828,0.920434,0.902409,0.901652
6,0.067300,0.059522,0.990072,0.920634,0.908029,0.905146
7,0.051400,0.053832,0.990736,0.927285,0.908439,0.909967
8,0.052200,0.051810,0.990948,0.926575,0.910278,0.912302
9,0.047800,0.049212,0.991085,0.935222,0.910492,0.914794
10,0.047800,0.046301,0.991443,0.933451,0.915289,0.918463


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 64 worker processes in total. Our suggested max number of worker in current system is 12, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 64 worker processes in total. Our suggested max number of worker in current system is 12, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarnin

TrainOutput(global_step=27420, training_loss=0.07269379259851359, metrics={'train_runtime': 16785.0763, 'train_samples_per_second': 1061.462, 'train_steps_per_second': 1.634, 'total_flos': 5.1200089907871744e+17, 'train_loss': 0.07269379259851359, 'epoch': 15.0})

In [ ]:
!nvidia-smi

Thu Mar 20 14:33:39 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA L4                      Off |   00000000:00:03.0 Off |                    0 |
| N/A   65C    P0             32W /   72W |   20981MiB /  23034MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

## Student Model Saving

In [ ]:
from huggingface_hub import login


login(token="HUGGING_FACE_KEY")

trainer.push_to_hub()

tokenizer.push_to_hub(repo_id="gates04/Structured-FP16-KD-SD-NID", commit_message="Student Model Tokenizer")

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/5.37k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/76.2M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.11k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/gates04/Structured-FP16-KD-SD-new-NID/commit/d1c0354001b2267745c861c4c882164aa3ad9271', commit_message='Student Model Tokenizer', commit_description='', oid='d1c0354001b2267745c861c4c882164aa3ad9271', pr_url=None, repo_url=RepoUrl('https://huggingface.co/gates04/Structured-FP16-KD-SD-new-NID', endpoint='https://huggingface.co', repo_type='model', repo_id='gates04/Structured-FP16-KD-SD-new-NID'), pr_revision=None, pr_num=None)